## 1. Pattern Detection

### 1.0 Retrieve the names of all ast files

In [17]:
from code_analysis import AST, ASTReader
from pathlib import Path
import os

## Global variables
reader = ASTReader()

# Check if output file already exists, if so, delete and create new file
filename = "output.txt"
if os.path.exists(filename):
    os.remove(filename)

# Open in 'append' mode to avoid overwriting the whole file after each modification
output_file = open(filename, "a")


## Get all the filenames in a given directory that sastifies a condition (*.ast.json)
def get_ast_json_files(directory):
   directory = Path(directory)
   return [str(file) for file in directory.rglob('*.ast.json')]


### 1.1 Create visitor that will visit a given ast in search of calls to an SQL Database
_**Note:** In php, calls to an SQL db can be done in the following ways:_
```
    mysql_query ( * ) ;
    mysqli_query ( * ) ;
    $object - > execute () ;
    $object - > mysql - > exec ( * ) ;
```  

In [18]:
class ASTSqlCallVisitor:
    # Not sure if this is necessary
    def __init__(self):
        self.ast = None
        self.isFirstVisitToFile = True

    def visit(self, ast: AST):
        self.ast = ast
        self.isFirstVisitToFile = True

        # State which file is being visited
        print(f"----- Visiting AST from file {self.ast.get_filename()} -----\n")

        # Start by visiting the root node of the ast
        # The private 'visit function' will recursively visit the children of each node
        self.__visit(self.ast.get_root())
        
    def __visit(self, node_id: int):
        # Define the conditions that must be respected for a node to be considered a call to an SQL DB (based on the above note)
        nodeImage = self.ast.get_image(node_id)
        isMethodOrFunctionCall = (self.ast.get_type(node_id) == "FunctionCall" or self.ast.get_type(node_id) == "MethodCall")

        isSqlQuery = isMethodOrFunctionCall and nodeImage == "mysql_query"
        isSqliQuery = isMethodOrFunctionCall and nodeImage == "mysqli_query"
        isExecuteMethodCall = isMethodOrFunctionCall and nodeImage == "execute"
        isExecMethodCall = isMethodOrFunctionCall and nodeImage == "exec"

        if isSqlQuery or isSqliQuery or isExecuteMethodCall or isExecMethodCall:
            # State which file is being visited - only printed in the output file if a call to db has been detected.
            if self.isFirstVisitToFile:
                output_file.write(f"\n")
                output_file.write(f"----- Visiting AST from file {self.ast.get_filename()} -----\n")
                self.isFirstVisitToFile = False

            output_file.write(f"Function '{nodeImage}' is called "
                f"at line {self.ast.get_position(node_id)[0]}\n")
            
        for child_id in self.ast.get_children(node_id):
            self.__visit(child_id)

### 1.2 For each ast file, load ast in memory and visit nodes in search of db calls

In [19]:
def findSqlInFolder(directory: str):
    # Retrieve filenames
    astFilenames = get_ast_json_files(directory)

    # Iterate over the filenames array
    for filename in astFilenames:
        # Load ast in memory
        ast = reader.read_ast(filename)

        # Visit ast
        visitor = ASTSqlCallVisitor()
        visitor.visit(ast)


### 1.3 Specify directory to analyze

In [ ]:
findSqlInFolder("../code_to_analyze/wordpress_ast/")
output_file.close()